In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
def f1(row, data, param):
    if param['state'] == '' or row[0].find('edit') > 0:
        param['state'] = row[0].split('[')[0]
    else:
        town = row[0].split('(')[0].strip()
        data.append([param['state'], town])

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    
    university_towns = pd.read_csv('university_towns.txt', header = None, delimiter = '\n')
    param = {'state': '' }
    data = []
    university_towns.apply(lambda row: f1(row, data, param), axis=1)
    return pd.DataFrame(data, columns=["State", "RegionName"])
get_list_of_university_towns()

In [ ]:
def get_GDP():
    GDP_raw = pd.read_excel('gdplev.xls', skiprows = 7, names = ['Year', 'GDP_cur_yearly','GDP_chained_yearly', 
                                                                  'Unnamed', 'Quarter', 'GDP_curr_quarterly',
                                                                   'GDP_chained_quarterly', 'unnamed'])
    GDP_yearly = GDP_raw[['Year', 'GDP_cur_yearly', 'GDP_chained_yearly']].dropna()
    GDP_quarterly = GDP_raw[['Quarter', 'GDP_curr_quarterly','GDP_chained_quarterly']]
    
    
    return GDP_yearly,GDP_quarterly
get_GDP()

In [ ]:
def get_housing_data():
    housing_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    return housing_data.dropna()
get_housing_data()

In [ ]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP_yearly, GDP_quarterly = get_GDP()
    
    for index in range(0,(GDP_quarterly.shape[0] - 3)):
        if (GDP_quarterly['GDP_curr_quarterly'].iloc[index] > GDP_quarterly['GDP_curr_quarterly'].iloc[index + 1] and
                       GDP_quarterly['GDP_curr_quarterly'].iloc[index + 2] > GDP_quarterly['GDP_curr_quarterly'].iloc[index + 1] and
                       GDP_quarterly['GDP_curr_quarterly'].iloc[index + 3] > GDP_quarterly['GDP_curr_quarterly'].iloc[index + 2]):
           
            key = GDP_quarterly['Quarter'].iloc[index]
    return key
get_recession_start()

In [ ]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    GDP_yearly, GDP_quarterly = get_GDP()
    
    for index in range(0,(GDP_quarterly.shape[0] - 3)):
        if (GDP_quarterly['GDP_curr_quarterly'].iloc[index] > GDP_quarterly['GDP_curr_quarterly'].iloc[index + 1] and
                       GDP_quarterly['GDP_curr_quarterly'].iloc[index + 2] > GDP_quarterly['GDP_curr_quarterly'].iloc[index + 1] and
                       GDP_quarterly['GDP_curr_quarterly'].iloc[index + 3] > GDP_quarterly['GDP_curr_quarterly'].iloc[index + 2]):
           
            key = GDP_quarterly['Quarter'].iloc[index + 3]
    return key
get_recession_end()

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3''' 
    GDP_yearly, GDP_quarterly = get_GDP()
    
    for index in range(0,(GDP_quarterly.shape[0] - 3)):
        if (GDP_quarterly['GDP_curr_quarterly'].iloc[index] > GDP_quarterly['GDP_curr_quarterly'].iloc[index + 1] and
                       GDP_quarterly['GDP_curr_quarterly'].iloc[index + 2] > GDP_quarterly['GDP_curr_quarterly'].iloc[index + 1] and
                       GDP_quarterly['GDP_curr_quarterly'].iloc[index + 3] > GDP_quarterly['GDP_curr_quarterly'].iloc[index + 2]):
           
            key = GDP_quarterly['Quarter'].iloc[index + 1]
    return key
get_recession_bottom()

In [ ]:
def quarters():
    #generating the new coloumns names 
    years = list(range(2000,2017))
    quarters = ['q1','q2','q3','q4']
    quarter_list = list()
    for i in years:
        for x in quarters:
            quarter_list.append((str(i) + x))
    return quarter_list

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    df_house = get_housing_data()
    
    qs = [list(df_house.columns)[x:x+3] for x in range(0, len(list(df_house.columns)), 3)]
        
    column_names = quarters()
    for col,q in zip(column_names,qs):
        df_house[col] = df_house[q].mean(axis=1)
        
    df_house = df_house[column_names]
 
    return df_house
convert_housing_data_to_quarters()

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"